In [1]:
'''script to regrid CMIP6 datatsets to target grid and store them'''

import numpy as np
import xarray as xr
import dask
from dask_gateway import Gateway
from distributed import worker_client, as_completed
import intake
import pandas as pd
import os
from collections import defaultdict
from tqdm.autonotebook import tqdm
from xmip.utils import google_cmip_col
from xmip.postprocessing import combine_datasets, _match_datasets,_concat_sorted_time
from cmip_catalogue_operations import reduce_cat_to_max_num_realizations, drop_vars_from_cat, drop_older_versions
from cmip_ds_dict_operations import select_period, drop_duplicate_timesteps, drop_coords, drop_incomplete
import xesmf as xe
from typing import Dict
import gcsfs
fs = gcsfs.GCSFileSystem() #list stores, stripp zarr from filename, load 

/tmp/ipykernel_6272/3747057756.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
# create regridders per source_id (datasets of source_id should be on the same grid after passing through reduce_cat_to_max_num_realizations
def create_regridder_dict(dataset_dict: Dict[str, xr.Dataset], target_grid_ds: xr.Dataset) -> Dict[str, xe.Regridder]:
    regridders = {}
    source_ids = np.unique([ds.attrs['source_id'] for ds in dataset_dict.values()])
    for si in tqdm(source_ids):
        matching_keys = [k for k in dataset_dict.keys() if si in k]
        # take the first one (we don't really care here which one we use)
        ds = dataset_dict[matching_keys[0]]
        regridder = xe.Regridder(ds,target_grid_ds,'bilinear',ignore_degenerate=True,periodic=True) #create regridder for this source_id
        regridders[si] = regridder
    return regridders

def make_filepath(output_path, ds):
    key = ds.attrs["ddict_key"]
    variable = ds.attrs['variable_id']
    return os.path.join(output_path,variable,ds.source_id,key+'.hist_'+ssp)

def pr_flux_to_m(ds):
    if 'pr' in ds.variables: #if dataset contains precipitation
        if ds.pr.units == 'kg m-2 s-1': #if precipitation units are flux 'kg m-2 s-1', convert to daily accumulated 'm'
            with xr.set_options(keep_attrs=True):
                ds['pr'] = 24*3600*ds['pr']/1000 #multiply by number of seconds in a day to get to kg m-2, and divide by density (kg/m3) to get to m    
                ds.pr.attrs['units'] = 'm'#convert pr units to m if
        elif ds.pr.units == 'm':
            ds = ds
        else:
            raise('Variable pr has unrecognized unit.')
    return ds 

def drop_duplicate_timesteps_from_ds(ds):
    #select only unique timesteps
    unique_time, idx = np.unique(ds.time,return_index=True)

    if len(ds.time) != len(unique_time):
        ds = ds.isel(time=idx)
    return ds

In [3]:
#configure settings
# output_path = 'gs://leap-persistent/timh37/CMIP6/timeseries_eu_1p5/'
output_path = 'gs://leap-scratch/timh37/CMIP6/datasets_eu_1p5/'
overwrite_existing = True #whether or not to process files for which output already exists (to-do: implement)

target_grid = xr.Dataset( #grid to interpolate CMIP6 simulations to
        {   "longitude": (["longitude"], np.arange(-30,22.5,1.5), {"units": "degrees_east"}),
            "latitude": (["latitude"], np.arange(70,30,-1.5), {"units": "degrees_north"}),})

query_vars = ['sfcWind','pr','psl'] #variables to process
required_vars = ['sfcWind','pr','psl'] #variables that includes models should provide

ssps = ['ssp245','ssp585']

In [4]:
#query simulations & manipulate data catalogue:
col = google_cmip_col() #google cloud catalog
qc_col = intake.open_esm_datastore("https://storage.googleapis.com/leap-persistent-ro/data-library/catalogs/cmip6-test/leap-pangeo-cmip6-test.json") #temporary pangeo-leap-forge catalogue
noqc_col = intake.open_esm_datastore("https://storage.googleapis.com/leap-persistent-ro/data-library/catalogs/cmip6-test/leap-pangeo-cmip6-noqc-test.json")

col_df = col.df
qc_df = qc_col.df
nonqc_df = noqc_col.df

#assign priority for keeping duplicate datasets in these catalogs
col_df['prio'] = 2
qc_df['prio'] = 3
nonqc_df['prio'] = 1

col.esmcat._df = pd.concat([col_df,qc_df,nonqc_df],ignore_index=True) #merge these catalogs
ssp_cats = defaultdict(dict)

#search catalog per ssp (need to do this for each SSP separately as availability may differ between them)
for s,ssp in enumerate(ssps):
    ssp_cat = col.search( #find instances providing all required query_vars for both historical & ssp experiments
    experiment_id=['historical',ssp],
    table_id='day',
    variable_id=required_vars,
    require_all_on=['source_id', 'member_id','grid_label'])
    ssp_cats[ssp] = ssp_cat
    
ssp_cats_merged = ssp_cats[ssp] #merge catalogues for all ssps, and drop duplicate historical simulations
ssp_cats_merged.esmcat._df = pd.concat([v.df for k,v in ssp_cats.items()],ignore_index=True).drop_duplicates(ignore_index=True)
ssp_cats_merged = drop_older_versions(ssp_cats_merged) #if google cloud and leap-pangeo catalogs provide duplicate datasets, keep the newest version, and if the versions are identical, keep the highest priority catalog
ssp_cats_merged = reduce_cat_to_max_num_realizations(ssp_cats_merged) #per model, select grid and 'ipf' combination providing most realizations (needs to be applied to both SSPs together to ensure the same variants are used under both scenarios)

In [5]:
#open datasets in dictionary
ssp_cats_merged.esmcat.aggregation_control.groupby_attrs = []
ddict_all = ssp_cats_merged.to_dataset_dict(zarr_kwargs={'use_cftime':True},aggregate=True) # single stores (Perhaps we dont need some of them, but at this point we do not really care)

#create xesmf regridders for each unique CMIP6 model
regridder_dict = create_regridder_dict(ddict_all, target_grid)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.member_id.table_id.variable_id.grid_label.zstore.dcpp_init_year.version.prio'


  0%|          | 0/28 [00:00<?, ?it/s]

In [6]:
## start new dask cluster
gateway = Gateway()

# close existing clusters (be careful if you have multiple clusters/servers open!)
open_clusters = gateway.list_clusters()
print(list(open_clusters))
if len(open_clusters)>0:
    for c in open_clusters:
        cluster = gateway.connect(c.name)
        cluster.shutdown()  


options = gateway.cluster_options()
options.worker_memory = 18
# options.worker_cores = 12

# Create a cluster with those options
cluster = gateway.new_cluster(options)
client = cluster.get_client()
cluster.adapt(20, 100)
client

[ClusterReport<name=prod.593d3a0b876a430c83e81dfcfd87fc2c, status=RUNNING>]


Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: /services/dask-gateway/clusters/prod.6d206a3df0fd4a0882539666f28e9840/status,


Regridding step:

In [7]:
regridded_datasets = []
for key,ds in tqdm(ddict_all.items()):
    ds.attrs["ddict_key"] = key #add current key information to attributes
    output_fn = make_filepath(output_path, ds)
    
    ds = ds.isel(dcpp_init_year=0,drop=True) #remove this coordinate

    regridder = regridder_dict[ds.attrs['source_id']] #select regridder for this source_id
    regridded_ds = regridder(ds, keep_attrs=True) #do the regridding
    
    regridded_datasets.append(regridded_ds.unify_chunks().chunk({'time':40000})) #append to list

  0%|          | 0/2373 [00:00<?, ?it/s]

Save regridder datasets to zarr in parallel in batches using dask cluster:

In [8]:
# following https://stackoverflow.com/questions/66769922/concurrently-write-xarray-datasets-to-zarr-how-to-efficiently-scale-with-dask
def write_wrapper(ds, overwrite=overwrite_existing, fs=None): #wrapper around to_zarr to submit to dask distributed cluster
    target = make_filepath(output_path, ds)
    with worker_client() as client:
        try:
            if overwrite or not fs.exists(target): # only write if store doesnt exist or overwrite is true
                ds = ds.drop_dims(['bnds','nbnd','height'],errors="ignore") #drop some auxiliary coordinates
                ds = pr_flux_to_m(ds) #convert pr units to m if pr in ds
                ds = drop_duplicate_timesteps_from_ds(ds)          
                
                ds.to_zarr(store=target, mode='w') #store
                return target, 'written freshly'
            else:
                return target, 'already written, skipped'
        except Exception as e:
            return target, f"Failed with: {e}"

# There is some more advanced way of doing this with the `as_completed` iterator, to achieve a 'steady' supply of submissions to the client. # (see answers in https://stackoverflow.com/questions/66769922/concurrently-write-xarray-datasets-to-zarr-how-to-efficiently-scale-with-dask), 
# but for our intents and purposes, we can just submit medium sized batches here:
# This seems to scale ok (there is still downtime between the batch submissions). For comparison, just using a big cluster and looping over the datasets achieved ~3x speed up (not bad), 
# but here we are looking at 10+x

interval = 50 # this seems to work fine, except a few warnings about a large graph... You could play with this, but higher numbers seemed to 
# make the scheduler quite unstable...
regridded_datasets_batches = [regridded_datasets[a:a+interval] for a in range(0,len(regridded_datasets), interval)]

written_stores = []

for ds_batch in tqdm(regridded_datasets_batches):
    # futures = [client.submit(write_wrapper, ds) for ds in ds_batch]
    futures = client.map(write_wrapper, ds_batch, overwrite=False, fs=fs)
    for future, result in as_completed(futures, with_results=True):
        written_stores.append(result)
        future.release()
    # do we need to deal with failed futures?
    # explicitly delete futures to ease pressure on client (JB: I do not 100% understand how this works TBH).
    del futures

  0%|          | 0/48 [00:00<?, ?it/s]

/srv/conda/envs/notebook/lib/python3.10/site-packages/distributed/client.py:3141: UserWarning: Sending large graph of size 187.69 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.10/site-packages/distributed/client.py:3141: UserWarning: Sending large graph of size 188.98 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.10/site-packages/distributed/client.py:3141: UserWarning: Sending large graph of size 186.67 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.10/site-packages/distributed/client.py:3141: UserWarning: Sending large graph of size 168.37 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.10/

In [8]:
for s,ssp in tqdm(enumerate(['ssp245','ssp585'])): #for each ssp:  
    #select historical and ssp data in merged catalogue for this particular ssp
    cat_to_open = ssp_cats_merged.search(
    experiment_id=['historical',ssp],
    table_id='day',
    variable_id=required_vars,
    require_all_on=['source_id', 'member_id','grid_label'])

    cat_to_open = drop_vars_from_cat(cat_to_open,[k for k in required_vars if k not in query_vars]) #out of required variables only process query variables
    #open datasets into dictionary
    cat_to_open.esmcat.aggregation_control.groupby_attrs = [] #to circumvent aggregate=false bug # @jbusecke: Which bug are you referring to? Issue?

    #to avoid this issue: https://github.com/intake/intake-esm/issues/496
        #doesn't actually aggregate if we set cmip6_cat.esmcat.aggregation_control.groupby_attrs = []
    kwargs = {'zarr_kwargs':{'consolidated':True,'use_cftime':True},'aggregate':True} #keyword arguments for generating dictionary of datasets from cmip6 catalogue
    # @jbusecke: Curious why you are not using xMIP here?
    ddict = cat_to_open.to_dataset_dict(**kwargs) #open datasets into dictionary
    #

    #preprocess datasets in dictionary
    ddict = pr_flux_to_m(ddict) #convert pr flux to accumulated pr
    ddict = drop_duplicate_timesteps(ddict) #remove duplicate timesteps if datasets have them
    #ddict = select_period(ddict,1850,2100) #preselect time periods, do this at later stage in the chain?
    ddict = drop_coords(ddict,['bnds','nbnd','height']) #remove some unused auxiliary coordinates
    
    break

0it [00:00, ?it/s]


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.member_id.table_id.variable_id.grid_label.zstore.dcpp_init_year.version'



KeyboardInterrupt



Calculate total size of datasets

In [ ]:
x=0
for k,v in ddict.items():
    if 'ssp245' in k:
        x += v.nbytes/1000000000
x